In [ ]:
import os
import time
from xpander_sdk import XpanderClient, LLMProvider, OpenAISupportedModels
from openai import OpenAI
from typing import List, Dict, Any
import logging
from dotenv import load_dotenv
from datetime import datetime
import logging
import re
import json

# Load environment variables
load_dotenv()
# Setup Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

# Load environment variables
OpenAPIKey = os.environ.get("OPENAI_API_KEY", "")
xpanderAPIKey = os.environ.get("XPANDER_API_KEY", "")
xpanderAgentID = os.environ.get("XPANDER_AGENT_ID", "")

openai_client = OpenAI(api_key=OpenAPIKey)
xpander_client = XpanderClient(api_key=xpanderAPIKey, base_url="https://inbound.stg.xpander.ai")
xpander_agent = xpander_client.agents.get(agent_id=xpanderAgentID)

class SharedMemory:
    def __init__(self):
        self.memory = []

    def add_message(self, content: str, role: str = "assistant", agent_name: str = None):
        self.memory.append({"role": role, "content": content, "agent_name": agent_name})

    def get_memory(self) -> list:
        return self.memory


class PlannerAgent:
    def __init__(self, handler, tools: list, task_message: str, system_message: str, shared_memory: SharedMemory,
                 finish_message: str = "Final Answer"):
        self.handler = handler
        self.tools = tools
        self.task_message = task_message
        self.system_message = system_message
        self.shared_memory = shared_memory
        self.local_memory = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": task_message},
        ]
        self.finish_message = finish_message
        self.is_finished = False
        self.step_number = 1
    def invoke_llm(self, memory, tools=None, model=OpenAISupportedModels.GPT_4_O, max_tokens=16384):
        try:
            response = self.handler.chat.completions.create(
                model=model,
                messages=memory,
                tools=tools,
                tool_choice="none",
                max_tokens=max_tokens,
                temperature=0.0, 
            )
            return response
        except Exception as e:
            raise RuntimeError(f"Error invoking LLM: {e}")
        
    def run_post_processing(self, response: str) -> str:
        """
        Post-process the LLM response to check for completion.
        """
        if re.search(self.finish_message, response):
            self.is_finished = True 
        return response

    def finished(self) -> bool:
        """
        Check if the agent has completed its task.
        """
        return self.is_finished



class ToolSelectorAgent:
    def __init__(self, handler, tools: list, task_message: str, system_message: str, shared_memory: SharedMemory):
        self.handler = handler
        self.tools = tools
        self.task_message = task_message
        self.system_message = system_message
        self.shared_memory = shared_memory
        self.local_memory = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": task_message},
        ]
        self.selected_tools = []

    def invoke_llm(self, memory, tools=None, model=OpenAISupportedModels.GPT_4_O, max_tokens=16384):
        try:                                
            response=self.handler.chat.completions.create(
                model=model,
                messages=memory,
                tools=tools,
                parallel_tool_calls=False,
                tool_choice="required",
                max_tokens=max_tokens,
                temperature=0.0,
            )
            return response
        except Exception as e:
            raise RuntimeError(f"Error invoking LLM: {e}")

class ParserAgent:
    def __init__(self, handler, tools: list, task_message: str, system_message: str, shared_memory: SharedMemory):
        self.handler = handler
        self.task_message = task_message
        self.system_message = system_message
        self.tools = tools
        self.shared_memory = shared_memory
        self.local_memory = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": task_message},
        ]
    def invoke_llm(self, memory, tools=None, model=OpenAISupportedModels.GPT_4_O, max_tokens=16384):
        try:
            response=self.handler.chat.completions.create(
                model=model,
                messages=memory,
                max_tokens=max_tokens,
                temperature=0.0,
                tools=tools,
                tool_choice="none"
            )
            return response
        except Exception as e:
            raise RuntimeError(f"Error invoking LLM ParserAgent: {e}")

In [2]:
system_prompt_planner = '''As a planner agent your task is to breakdown the main task into sub tasks that fulfill the main task. 
your main task is to retrieve only unread messages from Slack and from Gmail from the last 24 hours (today is {current_date}) and create a report in Notion.
'''
tool_selector_system_prompt = '''You are an tool selector agent responsible for selecting the correct tool with the most relevant parameters that will fullfil the task your will received from the planner agent to execute the plan.
'''
parser_system_prompt = '''You are a parser agent. Your job is to process raw outputs from tool calls and extract actionable information that can help fulfill the user request. You must ensure the extracted data is relevant, accurate, and formatted for easy use by the Planner Agent.
'''
planner_task_prompt = '''
As a planner research agent your task is to plan the the full workflow for the tool execution agent and the parser agent for fetching unread messages from Slack (only direct messages) and unread mails from Gmail from the last 24 hours,
and create a new page inside the 'daily reports' in Notion titled 'Daily Messages Summary - {current_date}' and populate it with the messages data.
you need to explain each step in the workflow, what is the current required data and what are the expected results.

what expected from you:
Retreive all the unread messeges from Slack - Only direct messages (DMs) from the last 24 hours, retrieve all unread mails from Gmail from the last 24 hours. 
after you finish to retrieve all the messages data from Slack and Gmail you need to create a new page inside the 'daily reports' database title as 'Daily Messages Summary - {current_date}' in Notion that contains all of messages data report.

rules:
1. you MUST said what is the current date in the response - today is {current_date}. it is very important for retreiving the correct messages from Gmail and Slack!!
2. You must retreive all the unread messages data from Slack (only direct messages) and Gmail when creating the final page!
3. on Slack You must go over each message in List User's Conversations and extract all the Unread direct messages data -  you MUST return the people names NOT ID.
4. The final answer must include a link to the new Notion page. 
5. after you finished to retrieve all the messages data from Slack and Gmail you must create the final page in Notion.
6. the new page in Notion must be inside the 'daily reports' page and titled "Daily Messages Summary - {current_date}" 
7. you must Populate the new page with "Daily Messages Summary - {current_date}" a table containing the following columns:
  - **Application**: The application that the message is from Gmail or Slack.
  - **Tag**: The message topic or category.
  - **Participants**: all the people included in each message.
  - **Response Needed**: Whether a response is expected.
  - **Urgency**: Highlights if the message is urgent (high, medium, low).
  - **Summary**: A brief overview of the message content.


this is the expected output template:
if the query has not been fulfilled:
Plan step (i+1): [the next step of your plan for how to solve the query].

if all the messages fetching from Slack and Gmail is finished and the page is created in Notion you'll return the final answer only block child has been appended at least 1 time in Notion.
you'll return a link to the page you need to return by the following template:
Final Answer: [link to the new page that created for Daily Messages Summary - {current_date}].

These are the strict rules:
1. Always provide your plan in natural language, ensuring it is closely related to the input tools, you must related to the available tools you got.
2. Be specific in the plan and explain what should be the results of this step after parsing the API response. 
3. Only if all unread messages data retreived from Slack and Gmail from the last 24 houres and the Notion page created with all information is ready and you got the link to this new page the user's query has been fulfilled and you need to return the output the answer immediately with the prefix: Final Answer: [link to the new page that created for the daily messages summary - {current_date}].
4. User's query can't be fulfilled without at least one 'Parser Response'. You can't fulfill task without at least one API calling and response.
5. If the query has not been fulfilled, explain how to fix the last step and continue to output your plan.
6. Return only the next Plan step (i+1) you generated and do not mention all the steps list until now. you'll get the conversation history Plan steps [1,...,i-1] and API responses after parsing, you will use it to generate the next step.
7. If the the API request failed, return how you recommend to handle this error in the next retry of the tool calling.
8. you must return only one step in each call! never return the full step pipeline in one iteration.
9. never create the final page before you featch all the messages data from Slack and Gmail if exist!
10. if there is no messages data from Slack or from Gmail you must return that you not found any messages and you are moving to the next step. also you must mention it in the final report in Notion.
11. all the retreived messages data can be founded in the previous steps after each prefix phrase: 
'here is the messages data that you retreive:'
[the resource tool Slack or Gmail response].
12. You will return the Final Answer only after fetchin all the messages data from Slack and Gmail and creating the final page in Notion. you can't return the final answer without using all tools and creating new blocks inside the page!
13. validate that you return all the people names or channel names NOT ID in the final report in Notion.

please start with the first step and return only one step in each call!
your daily messages summary report for {current_date} begin now...
'''
tool_selector_task_prompt = '''
as part of the multi agent pipeline you are the tool selector agent that will select the most accurate tool to fulfill the current task provided by the planner agent.
Your tasks:
1. Select the most accurate tool to fulfill the current task provided by the planner agent. 
2. Generate all required parameters by the schema you got that will fulfill the task.
3. You must return your answer as a tool_call with the function name and relevant arguments.
4. If the planer explain about the error and how to fix it, you should fix the last tool call parameters and return the new tool call.
5. when you creating the final Notion page you must used all the previews messages that collected in the parserAgent response steps.
6. you must fill all 'Daily Messages Summary - {current_date}' database table columns (properties) inside 'daily reports' by using the correct typing.
7. the new page blocks will includes the a table with the following columns: Application, Tag, Participants, Response Needed, Urgency, Summary.
8. you must use all the previews Parser Agent messages that include slack and gmail messages data.
'''

parser_task_prompt = '''
as part of the multi agent pipeline you are the parser agent and your task is to get the current step plan and the response from the tool that executed for this plan and parse this response to fulfill the plan.
your response should contains all the relevent details for the report without missing information.

These are the strict rules:
1. validate that you finished to extract all the unread direct messages (DMs) from Slack. you MUST go over each direct message in 'List User's Conversations' and extract all the Unread direct messages data. if you can't read a message continue to the next message in the list. you must ran over all the messages in the list.only when you finish to extract all the messages data you can tell the planner agent that you finished this step and all the messages data is ready.
2. you must validate that all the unread messages from Gmail retrieved successfully.count how many messages you get in List Messages, and than pay attention to extract all the messages data in the list. only when you finish to extract all the messages data you can tell the planner agent that you finished this step and all the messages data is ready.
3. after you finished to extract all the unread messages relevant data from Gmail you must Implement clustering logic to organize messages by related topics for easy viewing.
4. if there is no messages data from Slack or from Gmail you must return that you not found any messages and you can be moving to the next step.
5. you must fill all 'Daily Messages Summary - {current_date}' database table columns (properties) by using the correct typing.
6. only when 'The selected Tool:' is Notion create page it means that created a new page and you must return:
 page link: [link to this page]
 page id: [the new final page id]
 text: 'blocks not created yet, now need to add the information inside the blocks page.'
7. the new page blocks will MUST includes the a table with the following columns: 
  - **Application**: The application that the message is from Gmail or Slack.
  - **Tag**: The message topic or label (e.g: 'Meeting', 'Task', 'Event', 'Reminder', 'Notification', 'Alert', 'Transaction'...).
  - **Participants**: All the People included in each message. people names NOT ID separated by commas.
  - **Response Needed**: Whether a response is expected.
  - **Urgency**: Highlights if the message is urgent (high, medium, low).
  - **Summary**: A brief overview of the message content.
8. only when 'The selected Tool:' is 'append new block children' it means that blocks are created. you should return the page link with the message: 'block child has been appended in the [iter number (how many times the blocks created successfully)] time'.
9. you must return the report as human readable text that include all the information.

'''

In [ ]:
start_time = time.time()
tools = xpander_agent.get_tools()
current_date = datetime.now().date()
shared_memory = SharedMemory()

# Initialize Agents
planner_agent = PlannerAgent(handler=openai_client,tools=tools, task_message=planner_task_prompt.format(current_date=current_date), system_message=system_prompt_planner.format(current_date=current_date),shared_memory=shared_memory)
tool_selector_agent = ToolSelectorAgent(handler=openai_client,tools=tools,task_message=tool_selector_task_prompt.format(current_date=current_date),system_message=tool_selector_system_prompt , shared_memory=shared_memory)
parser_agent = ParserAgent(handler=openai_client, tools=tools, task_message=parser_task_prompt.format(current_date=current_date) ,system_message=parser_system_prompt, shared_memory=shared_memory)


planner_response = planner_agent.invoke_llm(memory=planner_agent.local_memory,tools=planner_agent.tools)
shared_memory.add_message(planner_response.choices[0].message.content,role="assistant",agent_name="PlannerAgent")
logger.info("Planner Response: %s", planner_response.choices[0].message.content)

tool_selector_response = tool_selector_agent.invoke_llm(memory=tool_selector_agent.local_memory+shared_memory.get_memory(),tools=tools)
tool_calls = xpander_client.extract_tool_calls(llm_response=tool_selector_response.model_dump(), llm_provider=LLMProvider.OPEN_AI)
_ = xpander_agent.run_tools(tool_calls)


In [ ]:
shared_memory = SharedMemory()
tools = xpander_agent.get_tools()

planner_response = planner_agent.invoke_llm(memory=planner_agent.local_memory,tools=tools)
shared_memory.add_message(planner_response.choices[0].message.content,role="assistant",agent_name="PlannerAgent")
logger.info("Planner Response: %s", planner_response.choices[0].message.content)

while not planner_agent.finished():
    try:

        # Step 2: Tool Selector Agent
        tool_selector_response = tool_selector_agent.invoke_llm(memory=tool_selector_agent.local_memory+shared_memory.get_memory(),tools=tools)
        tool_calls = xpander_client.extract_tool_calls(llm_response=tool_selector_response.model_dump(), llm_provider=LLMProvider.OPEN_AI)
        logger.info("Tool calls: %s", tool_calls)
        if tool_calls:
            for tool_call in tool_calls:
                tool_response = xpander_agent.run_tool(tool_call)
                logger.info("Tool response: %s", tool_response.result)
                
                selected_tool_params = json.dumps(tool_selector_response.model_dump()['choices'][0]['message']['tool_calls'])
                selected_tool_data = {
                    "selected_tool": tool_call.name,
                    "tool_call_id": tool_call.tool_call_id,
                    "params": selected_tool_params,
                    "response": tool_response.result
                }
                logger.info("Tool selector response: %s", json.dumps(selected_tool_data))
                
                parser_message = (
                    f"Current Task: {planner_response.choices[0].message.content}\n"
                    f"The selected Tool: {tool_call.name}\n"
                    f"The params Tool: {json.dumps(selected_tool_data)}\n"
                )

                parser_response = parser_agent.invoke_llm(memory=parser_agent.local_memory+shared_memory.get_memory()+[{"role": "user", "content": parser_message}],tools=tools)
                shared_memory.add_message(parser_response.choices[0].message.content,role="assistant",agent_name="ParserAgent")
                logger.info("Parser response: %s", parser_response.choices[0].message.content)


    except Exception as e:
        logger.error(f"Pipeline error: {e}")
        break
    
    tools = xpander_agent.get_tools()
    planner_response = planner_agent.invoke_llm(memory=planner_agent.local_memory+shared_memory.get_memory(),tools=tools)
    shared_memory.add_message(planner_response.choices[0].message.content,role="assistant",agent_name="PlannerAgent")
    logger.info("Planner response: %s", planner_response.choices[0].message.content)
    
    planner_agent.run_post_processing(planner_response.choices[0].message.content)

logger.info("Pipeline execution complete!")
logger.info("Shared Memory:")
logger.info(shared_memory.get_memory())

